In [1]:
import sympy as sp

In [1]:
import sympy as sp

# Define the dimensions
n = 2  # Example dimension for matrices

# Define real matrix symbols
U = sp.MatrixSymbol('U', n, n)
A = sp.MatrixSymbol('A', n, n)

# Convert MatrixSymbols to actual Matrices for operations
U = sp.Matrix(U, real=True)
A = sp.Matrix(A, real=True, symmetric=True)

# Perform matrix multiplication
UAU_T = U * A * U.T

# Extract diagonal elements
diag_UAU_T = sp.Matrix([UAU_T[i, i] for i in range(n)])

# Calculate Frobenius norm squared of the diagonal matrix
Err_U_A = diag_UAU_T.norm(2)**2

# Simplify the expression (optional)
Err_U_A_simplified = sp.simplify(Err_U_A)

# Compute the gradient of the error function with respect to each element of U
gradient_matrix = sp.Matrix(n, n, lambda i, j: Err_U_A.diff(U[i, j]))

print(gradient_matrix)

Matrix([[2*(((re(A[0, 0]*U[1, 0]) + re(A[1, 0]*U[1, 1]))*re(U[1, 0]) + (re(A[0, 1]*U[1, 0]) + re(A[1, 1]*U[1, 1]))*re(U[1, 1]) - (im(A[0, 0]*U[1, 0]) + im(A[1, 0]*U[1, 1]))*im(U[1, 0]) - (im(A[0, 1]*U[1, 0]) + im(A[1, 1]*U[1, 1]))*im(U[1, 1]))*((re(A[0, 0]*U[1, 0]) + re(A[1, 0]*U[1, 1]))*Derivative(re(U[1, 0]), U[0, 0]) + (re(A[0, 1]*U[1, 0]) + re(A[1, 1]*U[1, 1]))*Derivative(re(U[1, 1]), U[0, 0]) + (-im(A[0, 0]*U[1, 0]) - im(A[1, 0]*U[1, 1]))*Derivative(im(U[1, 0]), U[0, 0]) + (-im(A[0, 1]*U[1, 0]) - im(A[1, 1]*U[1, 1]))*Derivative(im(U[1, 1]), U[0, 0]) + (Derivative(re(A[0, 0]*U[1, 0]), U[0, 0]) + Derivative(re(A[1, 0]*U[1, 1]), U[0, 0]))*re(U[1, 0]) + (Derivative(re(A[0, 1]*U[1, 0]), U[0, 0]) + Derivative(re(A[1, 1]*U[1, 1]), U[0, 0]))*re(U[1, 1]) + (-Derivative(im(A[0, 0]*U[1, 0]), U[0, 0]) - Derivative(im(A[1, 0]*U[1, 1]), U[0, 0]))*im(U[1, 0]) + (-Derivative(im(A[0, 1]*U[1, 0]), U[0, 0]) - Derivative(im(A[1, 1]*U[1, 1]), U[0, 0]))*im(U[1, 1])) + ((re(A[0, 0]*U[1, 0]) + re(A[1, 0]

In [1196]:
import numpy as np
n = 10
A = np.random.rand(n, n)
A = A @ A.T  # Make A symmetric positive definite

A_bk = A.copy()
print(A)
U = np.eye(n)

def ddiag(U, A):
    return np.linalg.norm(np.diag(U @ A @ U.T), ord=2)**2

print(ddiag(U, A))
print(A[0, 0]**2 + A[1, 1]**2 + A[2, 2]**2)

[[3.33948432 2.35282377 3.48882558 1.54240798 3.18428211 2.58349248
  3.0732568  3.18769287 2.42850425 2.47889157]
 [2.35282377 2.79065949 3.12398825 1.89283863 2.80053389 2.5551883
  2.93482096 2.93714203 2.11385295 2.52431101]
 [3.48882558 3.12398825 4.73687638 1.91534314 3.69829836 3.30072376
  3.98541662 4.45825606 3.38940954 3.26963048]
 [1.54240798 1.89283863 1.91534314 1.6750627  2.05830836 1.52364404
  1.9597596  1.91501742 1.02902878 1.80703846]
 [3.18428211 2.80053389 3.69829836 2.05830836 4.12838648 3.31133184
  3.95948482 3.69124072 2.60362265 3.40561452]
 [2.58349248 2.5551883  3.30072376 1.52364404 3.31133184 3.48170489
  3.49299647 3.12920628 2.52427564 2.72423249]
 [3.0732568  2.93482096 3.98541662 1.9597596  3.95948482 3.49299647
  4.09489024 3.96171662 2.90527076 3.39120314]
 [3.18769287 2.93714203 4.45825606 1.91501742 3.69124072 3.12920628
  3.96171662 4.82582844 2.79943126 3.4445774 ]
 [2.42850425 2.11385295 3.38940954 1.02902878 2.60362265 2.52427564
  2.90527076 

In [1]:
from scipy.linalg import polar

def diag_norm(A):
    return np.linalg.norm(np.diag(A), ord=2)

def riemannian_diagonalization(A, U = None, h = 1e-2, iterations = 100, out = False):
    n = A.shape[1]
    if U == None:
        U = np.eye(n)

    A = A.copy()
    Utot = U.copy()

    d1 = diag_norm(A)
    it = 0
    finish = False
    while not finish:
        # x' = 1 + h * df/dx
        U = np.eye(n)
        for p in range(n):
            for q in range(n):
                U[p, q] -= h * A[p, p] * (A[p, q] + A[q, p])

        # projection to unitary space
        U, _ = polar(U, side = 'left')

        # A' = U^T A U
        A2 = U.T @ A @ U
        d2 = diag_norm(A2)
        if out:
            print(it, h, d1, d2)

        if d2 > d1: # accept step
            A = A2.copy()
            h *= 1.5
            Utot = Utot @ U
            d1 = d2
            it += 1
            if it >= iterations:
                finish = True
        else: # reject step
            h /= 2
            if h < 1e-16:
                finish = True

    # make sure numerical errors are corrected
    Utot, _ = polar(Utot, side = 'left')
    return np.diagonal(A), Utot

d, Utot = riemannian_diagonalization(A)

A2 = Utot.T @ A @ Utot
A2[np.abs(A2) < 1e-10] = 0

print(sorted(d))
A = A_bk.copy()
d2, U2 = np.linalg.eigh(A)
print(d2)
print(np.linalg.norm(d2 - sorted(d)))
print(Utot.T @ A @ Utot)


NameError: name 'A' is not defined

In [132]:
# Simultaneous diagonalization
from scipy.linalg import polar

def diag_norm_k(A):
    d = 0
    for i in range(A.shape[0]):
        d += np.linalg.norm(np.diag(A[i, :, :]), ord = 2)**2
    return d

A = np.random.rand(3, 3, 3)

def transform_matrices(A, U):
    A = A.copy()
    for k in range(A.shape[0]):
        A[k, :, :] = U.T @ A[k, :, :] @ U
    return A

def riemannian_joint_diagonalization(A, U = None, h = 1e-2, iterations = 100, out = False):
    if U == None:
        U = np.eye(A.shape[1])

    A = A.copy()
    Utot = U.copy()
    n = A.shape[1]

    d1 = diag_norm_k(A)
    it = 0
    finish = False
    while not finish:
        # x' = 1 + h * df/dx
        U = np.eye(n)
        for p in range(n):
            for q in range(n):
                for k in range(A.shape[0]):
                    U[p, q] -= h * A[k, p, p] * (A[k, p, q] + A[k, q, p])

        # projection to unitary space
        U, _ = polar(U, side = 'left')

        # A' = U^T A U
        A2 = transform_matrices(A, U)
        d2 = diag_norm_k(A2)
        if out:
            print(it, h, d1, d2)

        if d2 > d1: # accept step
            A = A2.copy()
            h *= 1.5
            Utot = Utot @ U
            d1 = d2
            it += 1
            if it >= iterations:
                finish = True
        else: # reject step
            h /= 2
            if h < 1e-16:
                finish = True

    # make sure numerical errors are corrected
    Utot, _ = polar(Utot, side = 'left')
    d = np.zeros([A.shape[1], A.shape[2]])
    for k in range(A.shape[0]):
        d[k, :] = np.diagonal(A[k, :, :])
    return d, Utot

d, Utot = riemannian_joint_diagonalization(A)
print(d)
A2 = transform_matrices(A, Utot)
print(A2)

[[ 1.37405573 -0.31600143  0.443884  ]
 [ 1.80052746  0.04790129  0.44581184]
 [ 1.54289007  0.1516515   0.55579786]]
[[[ 1.37405573 -0.15823836 -0.01111242]
  [-0.39450985 -0.31600143  0.44489375]
  [ 0.57661723 -0.31300013  0.443884  ]]

 [[ 1.80052746 -0.18456403 -0.45536415]
  [ 0.37260092  0.04790129 -0.40308586]
  [ 0.25287066  0.11259634  0.44581184]]

 [[ 1.54289007  0.26555096 -0.38388219]
  [ 0.16902552  0.1516515  -0.24489622]
  [ 0.12890965  0.28528213  0.55579786]]]


In [1193]:
# Weighted Simultaneous diagonalization
from scipy.linalg import polar
import numpy as np

def weighted_diagonal_norm(A, W):
    d = 0
    for k in range(A.shape[0]):
        for i in range(A.shape[1]):
            d += A[k, i, i]**2 / W[i, i]
    return d

def transform_matrices_w(A, W, U):
    A = A.copy()
    for k in range(A.shape[0]):
        A[k, :, :] = U.T @ A[k, :, :] @ U
    W = U.T @ W @ U
    return A, W

def weighted_diagonals(A, W):
    d = np.zeros([A.shape[1], A.shape[2]])
    for k in range(A.shape[0]):
        d[k, :] = np.diagonal(A[k, :, :]) / np.diagonal(W)
    return d

def weighted_symmetric(A, W):
    for k in range(A.shape[0]):
        A[k, :, :] = 0.5 * (A[k, :, :] @ W + W @ A[k, :, :])
    return A

def riemannian_joint_diagonalization(A, W, U = None, h = 1e-3, iterations = 1000, out = False):
    if U == None:
        U = np.eye(A.shape[1])

    A = A.copy()
    Utot = U.copy()
    n = A.shape[1]

    d1 = weighted_diagonal_norm(A, W)
    d0 = d1
    it = 0
    finish = False
    while not finish:
        # x' = 1 + h * df/dx
        G = np.zeros([n, n])
        G2 = np.zeros([n, n])
        for p in range(n):
            for q in range(n):
                for k in range(A.shape[0]):
                    G[p, q] += 2 * A[k, q, q] / W[q, q] * A[k, q, p] - A[k, q, q]**2 / W[q, q]**2 * W[q, p]
#                    G[p, q] += A[k, q, q] / W[q, q] * 2 * A[k, q, p] #- A[k, q, q]**2 / W[q, q]**2 * W[q, p] # like equations but seemingly problematic due to diagonal elements q, q
                    #G[p, q] += A[k, p, p] / W[p, p] * 2 * A[k, p, q] - A[k, p, p]**2 / W[p, p]**2 * W[p, q] # corrected version based on gut-feeling. multiplying second part with <1. seems to make it work
#                    G[p, q] += A[k, p, p] / W[p, p] * (2 * A[k, p, q] - 0.95 * A[k, p, p] * W[p, q] / W[p, p])
#                    G2[p, q] += A[k, p, p] * (A[k, p, q] + A[k, q, p])
                    G2[p, q] += A[k, q, q] * (A[k, q, p] + A[k, p, q])

#        if it % 1 == 0:
#            print(it, 'G-G2 = ', np.diag(G-G2)/np.diag(G))
#            print(it, 'G = ', G)
#            print(it, 'G2 = ', G2, '\n')
        U = np.eye(n)
        U += h * G

        # projection to unitary space
        U, _ = polar(U)
#        print('U=\n',U)
        U2 = np.eye(n) + h * G2
        U2, _ = polar(U2)
        U = U2
#        print('U2=\n',U2)

        # A' = U^T A U
        A2, W2 = transform_matrices_w(A, W, U)
        d2 = weighted_diagonal_norm(A2, W2)
        if out:
            print(it, h, d1, d2)

        if d2 > d1: # accept step
            A = A2.copy()
            W = W2.copy()
            if h < 1e1:
                h *= 1.5
            Utot = Utot @ U
            d1 = d2
            it += 1
            if it >= iterations:
                finish = True
        else: # reject step
            h /= 2
            if h < 1e-15:
                finish = True

    # make sure numerical errors are corrected
    Utot, _ = polar(Utot, side = 'left')
    d = weighted_diagonals(A, W)
    print('improvement: ', d1 / d0, '; d0 = ', d0, '; d1 = ', d1)
    print('convergence information: h = ', h, ', it = ', it)
    return d, Utot

def weighted_diag(A, W, U = None, h = 1e-1, micro_iterations = 100, macro_iterations = 3, out = False):
    Utot = np.eye(A.shape[1])
    for it in range(macro_iterations):
        d, U = riemannian_joint_diagonalization(A, W, h = 1e-2, iterations = micro_iterations, out = False)
        Utot = Utot @ U
        A, W = transform_matrices_w(A, W, U)
    return d, Utot
    

#def riemannian_joint_diagonalization(A, W, U = None, h = 1e-1, iterations = 1000, out = False):

def initial_A(n):
    A = np.random.rand(n, n, n)
    for k in range(n):
        A[k, :, :] = A[k, :, :] @ A[k, :, :].T
    A[1, :, :] = A[0, :, :] @ A[0, :, :]
    A[2, :, :] = A[0, :, :] @ A[1, :, :]
    for k in range(n):
        A[k, :, :] /= np.trace(A[k, :, :])
    return A

def initial_W(n):
    W = np.random.rand(n, n)
    W = W @ W.T
    for i in range(n):
        W[i, i] *= 5
    W /= np.trace(W)
    return W

def report(A, W, pre = None):
    print(pre + 'A =\n', A)
    print(pre + 'W =\n', W, np.trace(W))
    d = weighted_diagonals(A, W)
    print(pre + 'd =\n', d)

#W = initial_W(3)
W = np.eye(3)
A = initial_A(3)
A = weighted_symmetric(A, W)

report(A, W, pre = 'start ')

#d, Utot = riemannian_joint_diagonalization(A, W, out = False)
d, Utot = weighted_diag(A, W, macro_iterations = 5, micro_iterations=200)

A2, W2 = transform_matrices_w(A, W, Utot)

report(A2, W2, pre = 'end ')

start A =
 [[[0.44558043 0.30848414 0.27979883]
  [0.30848414 0.32009358 0.23387717]
  [0.27979883 0.23387717 0.23432599]]

 [[0.45800105 0.37137916 0.32305107]
  [0.37137916 0.31066072 0.26591683]
  [0.32305107 0.26591683 0.23133822]]

 [[0.45854535 0.37635656 0.32589576]
  [0.37635656 0.30963198 0.26779664]
  [0.32589576 0.26779664 0.23182267]]]
start W =
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 3.0
start d =
 [[0.44558043 0.32009358 0.23432599]
 [0.45800105 0.31066072 0.23133822]
 [0.45854535 0.30963198 0.23182267]]
improvement:  2.6004600136794096 ; d0 =  1.0755804627529781 ; d1 =  2.7970039848839154
convergence information: h =  14.778918800354008 , it =  200
improvement:  1.0000001381582875 ; d0 =  2.7970039848839017 ; d1 =  2.7970043713131822
convergence information: h =  14.778918800354008 , it =  200
improvement:  1.0000000002764071 ; d0 =  2.7970043713131822 ; d1 =  2.797004372086294
convergence information: h =  14.778918800354008 , it =  200
improvement:  1.0000000000000002 ; 